In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# Initialize LLM
llm = ChatOpenAI(temperature=0.1)

# Define the cache directory
cache_dir = LocalFileStore("./.cache")

# Initialize the text splitter
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=300,
    chunk_overlap=30,
)

# Load and split the document
loader = UnstructuredFileLoader("./files/BAU_Report_May.docx")

docs = loader.load_and_split(text_splitter=splitter)

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize cache-backed embeddings
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# Initialize the vector store using the cachedembeddings
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you dont know the answer, just say you don't know, don't make it up:\n\n{context}",
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

retriever = vectorstore.as_retriever()

# list of docs

# for doc in list of docs | prompt | llm

# for response in list of llm responses | put them all together

# final doc | prompt | llm

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    Use the following portion of a long document to see if any of the text is relevant to answer the question. Return an relevant text verbatim.
    -----
    {context}
    """,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    Given the following extracted parts of a long document and a question, creata a final answer. If you dont know the answer, just say that you dont know. Dont try to make up an answer.
    -----
    {context}
    """,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)


chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

# Initialize the retrieval chain
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

# Run the chain with a query
chain.invoke("Describe 1 acheivement")

AIMessage(content='One achievement highlighted in the report is the completion of the Canvas access improvements by the Resource Team.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 1123, 'total_tokens': 1142}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c7ddce2d-8051-4dd8-99d5-97ab721c3ba4-0', usage_metadata={'input_tokens': 1123, 'output_tokens': 19, 'total_tokens': 1142})